In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle 

In [5]:
df = pd.read_csv(r'C:\Users\Nu13359Ab\ChurnClassification\Churn_Modelling.csv') 
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Preprocess the data 
## Drop irrelevant columns 
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1) 
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
## Encode categorical variables 
label_encoder_gender = LabelEncoder() 
df['Gender'] = label_encoder_gender.fit_transform(df['Gender']) 
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [8]:
## Onehot encode 'Geography' 
from sklearn.preprocessing import OneHotEncoder 
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']]) 
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography'])) 
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
### Combine one hot encoder columns with the original df 
df = pd.concat([df.drop('Geography', axis = 1), geo_encoder_df], axis =1 ) 
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
## Split the data into features and target 
X = df.drop('EstimatedSalary', axis=1) 
y = df['EstimatedSalary']

In [12]:
## Split the data in graining and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 


In [13]:
## Scale these features 
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)

In [14]:
## Save the encoders and scalerr 
with open('label_encoder_gender.pkl', 'wb') as file: 
    pickle.dump(label_encoder_gender, file) 

with open('onehot_encoder_geo.pkl', 'wb') as file: 
    pickle.dump(onehot_encoder_geo, file) 

with open('scaler.pkl', 'wb') as file: 
    pickle.dump(scaler, file)

### ANN Regression Problem Satatement

In [16]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense  

In [20]:
## Build ANN Model  
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 Connected with input layer
    Dense(32, activation='relu'),  ## HL2 
    Dense(1) ## output layer for regression 
]) 

## compile the model 
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae']) 

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
## Set Up the Tensorboard  
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime  

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
tensorflow_callback = TensorBoard(log_dir= log_dir, histogram_freq=1)

In [22]:
## Set up Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 

In [23]:
# Train the model 
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100, 
    callbacks = [tensorflow_callback, early_stopping_callback]
) 

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 100379.8516 - mae: 100379.8516 - val_loss: 98526.6250 - val_mae: 98526.6250
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 99683.2422 - mae: 99683.2422 - val_loss: 97126.2344 - val_mae: 97126.2344
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 97251.8906 - mae: 97251.8906 - val_loss: 93554.5234 - val_mae: 93554.5234
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 92438.9844 - mae: 92438.9844 - val_loss: 87465.3906 - val_mae: 87465.3906
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 85214.2969 - mae: 85214.2969 - val_loss: 79330.2734 - val_mae: 79330.2734
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 76449.5391 - mae: 76449.5391 - val_loss: 70667.7734 - val_mae: 70667.7734
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 67732.9844 - mae: 67732.9844 - val_loss: 62949.8672 - val_mae: 62949.8672
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step 

In [24]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 25828), started 0:00:08 ago. (Use '!kill 25828' to kill it.)

In [28]:
## Evaluete model on the test data 
test_loss, test_mae = model.evaluate(X_test,y_test) 
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50177.0273 - mae: 50177.0273
Test MAE : 50177.02734375


In [29]:
model.save('regression_model.h5')